# original data GCN

In [ ]:
import numpy as np

pos1 = np.array([[0,2,0,1,0], [0,0,1,0,0],[0,0,0,0,0],[0,0,2,0,1],[0,0,0,0,0]])
pos2 = np.array([[0,3,0,2,0], [0,0,2,0,0],[0,0,0,0,0],[0,0,3,0,2],[0,0,0,0,0]])

print(np.mumal(pos1,pos2))

In [1]:
import sys
sys.path.append("../codes")

import os
import torch
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

NameError: name 'sys' is not defined

In [2]:
from set_data_folder import make_train_data
from model import GCN
from train_test import train, test
from my_data_train_utils import make_datasets, conf_matrix

from torch_geometric.utils import from_networkx
from torch_geometric.data import Data, DataLoader

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
node = "1000"
m_list = [2, 4, 6, 8, 10]
# p_list = [0.04, 0.08, 0.11, 0.15, 0.19]
p_list = [0.004, 0.008, 0.012, 0.016, 0.02]
# step_list = [200, 384, 564, 736, 900]
step_list = [1996, 3984, 5964, 7936, 9900]

for m, p, step in zip(m_list, p_list, step_list):
    # make train data folder
    p_s = [
        {"kind": "barabasi", "node": [node], "p": [str(m)]},
        {"kind": "noGrowth", "node": [node], "p": [str(step)]},
        {"kind": "noAttach", "node": [node], "p": [str(m)]},
        {"kind": "random", "node": [node], "p": [str(p)]}
    ]
    make_train_data(p_s, "../train_data/net").copy_data()
    
    # load data
    train_loader, test_loader = make_datasets(batch_size=100, split_rate=0.7)
    
    # set model
    model = GCN(hidden_channels=64, num_classes=4, num_node_feature=1)
    model.to(device)
    optimizer = torch.optim.Adagrad(model.parameters(), lr=0.05, eps=1e-5)
    criterion = torch.nn.CrossEntropyLoss()
    
    # train
    train_accs = []
    test_accs = []

    for epoch in range(1, 20):
        train(model, train_loader, criterion, optimizer, device)
        train_acc = test(model, train_loader, device)
        test_acc = test(model, test_loader, device)

        train_accs.append(train_acc)
        test_accs.append(test_acc)

        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    
    # make result save dir
    save_dir = "/workspace/4class_result/1000/" + "_".join([str(m),str(p),str(step)])
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    print(save_dir)
    
    # model(重み) の保存
    torch.save(model.to("cpu").state_dict(), save_dir + "/model.pth")
    # 学習結果の保存

    df = pd.DataFrame()
    df["train_acc"] = train_accs
    df["test_acc"] = test_accs
    df.to_csv(save_dir + "/log.csv")
    
    # 学習曲線の保存
    plt.style.use('ggplot')
    plt.plot(train_accs, label="train")
    plt.plot(test_accs, label="test")
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(save_dir + "/learning.png")
    plt.clf()
    plt.close()
    
    # make conf matrix
    conf_matrix(model, test_loader, device)
    plt.savefig(save_dir + "/conf_matrix.png")
    plt.clf()
    plt.close()

Epoch: 001, Train Acc: 0.2500, Test Acc: 0.2500
Epoch: 002, Train Acc: 0.8252, Test Acc: 0.8277
Epoch: 003, Train Acc: 0.7735, Test Acc: 0.7732
Epoch: 004, Train Acc: 0.8050, Test Acc: 0.8047
Epoch: 005, Train Acc: 0.9182, Test Acc: 0.9128
Epoch: 006, Train Acc: 0.8959, Test Acc: 0.8935
Epoch: 007, Train Acc: 0.9230, Test Acc: 0.9200
Epoch: 008, Train Acc: 0.9007, Test Acc: 0.9007
Epoch: 009, Train Acc: 0.9196, Test Acc: 0.9135
Epoch: 010, Train Acc: 0.9066, Test Acc: 0.9058
Epoch: 011, Train Acc: 0.9137, Test Acc: 0.9137
Epoch: 012, Train Acc: 0.9301, Test Acc: 0.9252
Epoch: 013, Train Acc: 0.9236, Test Acc: 0.9217
Epoch: 014, Train Acc: 0.9229, Test Acc: 0.9143
Epoch: 015, Train Acc: 0.9184, Test Acc: 0.9162
Epoch: 016, Train Acc: 0.9265, Test Acc: 0.9243
Epoch: 017, Train Acc: 0.9279, Test Acc: 0.9203
Epoch: 018, Train Acc: 0.9296, Test Acc: 0.9247
Epoch: 019, Train Acc: 0.9319, Test Acc: 0.9267
/workspace/4class_result/1000/2_0.004_1996
Epoch: 001, Train Acc: 0.5000, Test Acc: 0.50